In [1]:
import pandas as pd
import ast

def parse_assigned_value(value_str):
    """
    Safely convert the string in 'assigned_value' into a Python object.
    If it fails, return the original string.
    """
    try:
        return ast.literal_eval(value_str)
    except:
        return value_str

def parse_range_tuple(val):
    """
    If val is a 2-item tuple of floats, return (float1, float2).
    Otherwise return (None, None).
    """
    if isinstance(val, tuple) and len(val) == 2:
        try:
            return float(val[0]), float(val[1])
        except:
            return None, None
    return None, None

def flatten_hvac_data(df_input, out_build_csv, out_zone_csv):
    """
    Splits data into:
      1) building-level => param_name == "hvac_params"
         => each key => row
           If the value is a tuple => store param_value="range(x,y)",
                                     param_min=x, param_max=y
           Else => store param_value as is, param_min, param_max blank
      2) zone-level => param_name == "zones"
         => flatten zone data => (bldg_id, zone_name, param_name, param_value)

    Writes two CSVs.
    """
    building_rows = []
    zone_rows = []

    for idx, row in df_input.iterrows():
        bldg_id = row["ogc_fid"]
        main_pname = row["param_name"]   # "hvac_params" or "zones"
        assigned_val = row["assigned_value"]

        if main_pname == "hvac_params":
            # assigned_val is typically a dict of e.g. {"heating_day_setpoint": 20.3, "heating_day_setpoint_range":(19,21), ...}
            if isinstance(assigned_val, dict):
                for k, v in assigned_val.items():
                    param_value_str = v
                    param_min, param_max = None, None

                    # if v is a tuple => treat as range
                    if isinstance(v, tuple) and len(v) == 2:
                        parsed_min, parsed_max = parse_range_tuple(v)
                        if parsed_min is not None and parsed_max is not None:
                            param_value_str = f"range({parsed_min},{parsed_max})"
                            param_min, param_max = parsed_min, parsed_max

                    building_rows.append({
                        "ogc_fid": bldg_id,
                        "param_name": k,
                        "param_value": param_value_str,
                        "param_min": param_min,
                        "param_max": param_max
                    })
            else:
                # Unusual case if assigned_val is not a dict
                building_rows.append({
                    "ogc_fid": bldg_id,
                    "param_name": "unrecognized_hvac_params",
                    "param_value": assigned_val,
                    "param_min": None,
                    "param_max": None
                })

        elif main_pname == "zones":
            # assigned_val => dict of zone_name => { param_name => param_value, ... }
            if isinstance(assigned_val, dict):
                for zone_name, zone_dict in assigned_val.items():
                    for zparam_name, zparam_val in zone_dict.items():
                        zone_rows.append({
                            "ogc_fid": bldg_id,
                            "zone_name": zone_name,
                            "param_name": zparam_name,
                            "param_value": zparam_val
                        })
            else:
                # If it's not a dict => store raw
                zone_rows.append({
                    "ogc_fid": bldg_id,
                    "zone_name": "unrecognized_zone",
                    "param_name": "unrecognized_zparam",
                    "param_value": assigned_val
                })

        else:
            # If there's any other param_name, skip or store as you see fit
            pass

    # Convert to DataFrame
    df_build = pd.DataFrame(building_rows)
    df_zone = pd.DataFrame(zone_rows)

    # Ensure columns exist if empty
    if df_build.empty:
        df_build = pd.DataFrame(columns=["ogc_fid", "param_name", "param_value", "param_min", "param_max"])
    if df_zone.empty:
        df_zone = pd.DataFrame(columns=["ogc_fid", "zone_name", "param_name", "param_value"])

    # Reorder columns for building-level
    df_build = df_build[["ogc_fid", "param_name", "param_value", "param_min", "param_max"]]
    # zone-level is fine
    df_zone = df_zone[["ogc_fid", "zone_name", "param_name", "param_value"]]

    # Write to CSV
    df_build.to_csv(out_build_csv, index=False)
    df_zone.to_csv(out_zone_csv, index=False)

    print(f"[INFO] Wrote building-level picks to {out_build_csv}")
    print(f"[INFO] Wrote zone-level picks to {out_zone_csv}")


def main():
    # 1) Path to your original HVAC CSV
    csv_in = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_params.csv"
    # 2) Output CSVs
    csv_build_out = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_building.csv"
    csv_zone_out  = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_zones.csv"

    # 3) Load CSV
    df = pd.read_csv(csv_in)

    # 4) Parse the assigned_value from string to python object
    df["assigned_value"] = df["assigned_value"].apply(parse_assigned_value)

    # 5) Flatten + export
    flatten_hvac_data(
        df_input=df,
        out_build_csv=csv_build_out,
        out_zone_csv=csv_zone_out
    )

if __name__ == "__main__":
    main()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Documents\\E_Plus_2030_py\\output\\assigned\\assigned_hvac_params.csv'

In [16]:
import pandas as pd
import ast  # For safely parsing "(4.0, 5.0)" as a Python tuple

def transform_fenez_log_to_structured_with_ranges(
    csv_input="output/assigned/assigned_fenez_params.csv",
    csv_output="output/assigned/structured_fenez_params.csv",
):
    """
    Reads the 'flat' fenestration/material CSV (with param_name & assigned_value),
    and outputs a 'structured' CSV that:

      - Merges final assigned value + min/max range into one row per parameter.
      - Does NOT skip params that have empty or None values.
      - Always includes a row for any param that appears in the CSV, even if
        there's no final value or no range.

    Final columns:
      ogc_fid, sub_key, eplus_object_type, eplus_object_name,
      param_name, param_value, param_min, param_max
    """

    df = pd.read_csv(csv_input)

    # We'll keep track of data in a nested dict:
    # final_dict[(ogc_fid, sub_key)] = {
    #   "obj_type": <str or None>,
    #   "obj_name": <str or None>,
    #   "params": {
    #       "Thickness": {
    #          "value": <final assigned value or None>,
    #          "min": <float or None>,
    #          "max": <float or None>
    #       },
    #       "Conductivity": {...},
    #       etc.
    #   }
    # }

    final_dict = {}

    def get_subdict(fid, s_key):
        """Helper to retrieve or create the dictionary entry for (fid, s_key)."""
        if (fid, s_key) not in final_dict:
            final_dict[(fid, s_key)] = {
                "obj_type": None,
                "obj_name": None,
                "params": {}
            }
        return final_dict[(fid, s_key)]

    for i, row in df.iterrows():
        # Must have these columns to proceed
        if "ogc_fid" not in row or "param_name" not in row or "assigned_value" not in row:
            continue

        ogc_fid = row["ogc_fid"]
        param_name = str(row["param_name"])
        assigned_value = row["assigned_value"]

        # We only transform if param_name starts with "fenez_"
        # (Otherwise, skip non-fenestration logs.)
        if not param_name.startswith("fenez_"):
            continue

        # Remove the "fenez_" prefix => e.g. "doors_opq.Thermal_Resistance_range"
        remainder = param_name[len("fenez_"):]  # e.g. "doors_opq.Thermal_Resistance_range"

        if "." in remainder:
            sub_key, field = remainder.split(".", 1)
        else:
            # e.g. "wwr", "roughness" => treat them as sub_key=that word, no field
            sub_key = remainder
            field = None

        # Retrieve or create sub-dict for (ogc_fid, sub_key)
        subd = get_subdict(ogc_fid, sub_key)

        # (A) If this row indicates the E+ object type
        if field == "obj_type":
            # e.g. assigned_value = "MATERIAL:NOMASS"
            subd["obj_type"] = assigned_value

        # (B) If this row indicates the E+ object name
        elif field == "Name":
            subd["obj_name"] = assigned_value

        # (C) If the field ends with "_range", parse as min/max
        elif field and field.endswith("_range"):
            # e.g. field="Thermal_Resistance_range"
            base_param = field.replace("_range", "")  # "Thermal_Resistance"

            if base_param not in subd["params"]:
                subd["params"][base_param] = {"value": None, "min": None, "max": None}

            # assigned_value might be something like "(4.0, 5.0)" or maybe "None"
            try:
                maybe_tuple = ast.literal_eval(str(assigned_value))
                if isinstance(maybe_tuple, (list, tuple)) and len(maybe_tuple) == 2:
                    min_val, max_val = maybe_tuple
                    subd["params"][base_param]["min"] = min_val
                    subd["params"][base_param]["max"] = max_val
            except:
                pass  # If parse fails or it's "None", we leave them as None

        # (D) If it's a 'normal' field => param_name like "Thermal_Resistance"
        else:
            # If there's no field, we might be dealing with e.g. "roughness" or "wwr"
            # which are top-level. We'll store them as a param anyway, in case you want them.
            if field is None:
                # e.g. sub_key="wwr", so param_name = sub_key
                # We'll store it under subd["params"][sub_key] to keep it consistent
                p_name = sub_key
                if p_name not in subd["params"]:
                    subd["params"][p_name] = {"value": None, "min": None, "max": None}
                subd["params"][p_name]["value"] = assigned_value
            else:
                # e.g. field="Thermal_Resistance", assigned_value=4.23
                if field not in subd["params"]:
                    subd["params"][field] = {"value": None, "min": None, "max": None}
                subd["params"][field]["value"] = assigned_value

    # Now finalize. We'll produce a row for **every** param in subd["params"], even if
    # param_value is None and min/max are None.

    structured_rows = []
    for (fid, s_key), info in final_dict.items():
        obj_type = info["obj_type"]
        obj_name = info["obj_name"]
        params = info["params"]  # a dict => param_name => { value, min, max }

        # If no params => possibly skip if you want. But let's create NO row or a placeholder?
        # We do NOT skip them if you want a row. But let's skip if sub_key has no params at all.
        if not params:
            # Possibly store a row for "sub_key with no params"? If you want that, do:
            # structured_rows.append({ "ogc_fid": fid, "sub_key": s_key, ... })
            # But typically we skip if no actual param.
            continue

        # For each param in "params"
        for p_name, pvals in params.items():
            param_value = pvals["value"]
            param_min = pvals["min"]
            param_max = pvals["max"]

            structured_rows.append({
                "ogc_fid": fid,
                "sub_key": s_key,
                "eplus_object_type": obj_type,
                "eplus_object_name": obj_name,
                "param_name": p_name,
                "param_value": param_value,  # might be None or empty
                "param_min": param_min,      # might be None
                "param_max": param_max       # might be None
            })

    # Convert to DataFrame
    df_out = pd.DataFrame(structured_rows)

    # (Optional) attempt to float-convert param_value, param_min, param_max
    def try_float(x):
        try:
            return float(x)
        except:
            return x

    for col in ["param_value", "param_min", "param_max"]:
        df_out[col] = df_out[col].apply(try_float)

    df_out.to_csv(csv_output, index=False)
    print(f"[transform_fenez_log_to_structured_with_ranges] => wrote: {csv_output}")


if __name__ == "__main__":
    transform_fenez_log_to_structured_with_ranges(
        csv_input="output/assigned/assigned_fenez_params.csv",
        csv_output="output/assigned/structured_fenez_params.csv"
    )


[transform_fenez_log_to_structured_with_ranges] => wrote: output/assigned/structured_fenez_params.csv


In [2]:
###############################################################
# flatten_assigned_vent.py
###############################################################

import pandas as pd
import ast

def parse_assigned_value(value_str):
    """
    Safely convert the string in 'assigned_value' into a Python dict,
    e.g. literal_eval("{'infiltration_base':1.23}")
    """
    try:
        return ast.literal_eval(value_str)
    except:
        return {}

def flatten_ventilation_data(df_input, out_build_csv, out_zone_csv):
    """
    Takes a DataFrame with columns [ogc_fid, param_name, assigned_value].
    Splits it into two DataFrames:
      1) building-level (flattening 'building_params')
      2) zone-level (flattening 'zones').

    Then writes them to CSV.
    """
    building_rows = []
    zone_rows = []

    for idx, row in df_input.iterrows():
        bldg_id = row["ogc_fid"]
        param_name = row["param_name"]
        assigned_val = row["assigned_value"]  # should now be a dict

        if param_name == "building_params":
            # assigned_val is a dict of infiltration_base, year_factor, fan_pressure, etc.
            for k, v in assigned_val.items():
                building_rows.append({
                    "ogc_fid": bldg_id,
                    "param_name": k,
                    "param_value": v
                })

        elif param_name == "zones":
            # assigned_val is a dict: { "Zone1": {...}, "Zone2": {...}, ... }
            for zone_name, zone_dict in assigned_val.items():
                for zparam_name, zparam_val in zone_dict.items():
                    zone_rows.append({
                        "ogc_fid": bldg_id,
                        "zone_name": zone_name,
                        "param_name": zparam_name,
                        "param_value": zparam_val
                    })

        else:
            # If there's something else or you want to handle further data
            pass

    # Convert to DataFrame
    df_build = pd.DataFrame(building_rows)
    df_zone = pd.DataFrame(zone_rows)

    # Reorder columns if you like
    df_build = df_build[["ogc_fid", "param_name", "param_value"]]
    df_zone = df_zone[["ogc_fid", "zone_name", "param_name", "param_value"]]

    # Write to CSV
    df_build.to_csv(out_build_csv, index=False)
    df_zone.to_csv(out_zone_csv, index=False)

    print(f"[INFO] Wrote building-level picks to {out_build_csv}")
    print(f"[INFO] Wrote zone-level picks to {out_zone_csv}")


def main():
    # 1) Path to your original CSV
    csv_in = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_ventilation.csv"

    # 2) Paths for output
    csv_build_out = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_vent_building.csv"
    csv_zone_out = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_vent_zones.csv"

    # 3) Read the CSV
    df_assigned = pd.read_csv(csv_in)

    # 4) Convert 'assigned_value' from string to dict
    #    using 'ast.literal_eval' or a helper function
    df_assigned["assigned_value"] = df_assigned["assigned_value"].apply(parse_assigned_value)

    # 5) Flatten
    flatten_ventilation_data(
        df_input=df_assigned,
        out_build_csv=csv_build_out,
        out_zone_csv=csv_zone_out
    )

if __name__ == "__main__":
    main()


[INFO] Wrote building-level picks to D:\Documents\E_Plus_2030_py\output\assigned\assigned_vent_building.csv
[INFO] Wrote zone-level picks to D:\Documents\E_Plus_2030_py\output\assigned\assigned_vent_zones.csv


In [15]:
df_assigned = pd.read_csv(r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_ventilation.csv")
df_assigned

,ogc_fid,param_name,assigned_value
0,4136730,building_params,"{'infiltration_base': 1.2278853596915769, 'inf..."
1,4136730,zones,{'Zone1': {'infiltration_object_name': 'Infil_...
2,4136732,building_params,"{'infiltration_base': 0.6278853596915768, 'inf..."
3,4136732,zones,{'Zone1_FrontPerimeter': {'infiltration_object...


In [5]:
import pandas as pd
import ast

def parse_assigned_value(value_str):
    """
    Safely convert the string in 'assigned_value' into a Python dict.
    Uses ast.literal_eval to parse e.g.
      "{'heating_day_setpoint': 20.28, 'cooling_day_setpoint': 24.55, ...}"
    into a real Python dictionary.
    """
    try:
        return ast.literal_eval(value_str)
    except:
        return {}

def flatten_hvac_data(df_input, out_build_csv, out_zone_csv):
    """
    Takes a DataFrame with columns [ogc_fid, param_name, assigned_value].
    Splits it into two DataFrames:
      1) building-level (where param_name == "hvac_params")
      2) zone-level (where param_name == "zones").

    Then writes them to CSV (out_build_csv, out_zone_csv).
    """
    building_rows = []
    zone_rows = []

    for idx, row in df_input.iterrows():
        bldg_id = row["ogc_fid"]
        param_name = row["param_name"]
        assigned_val = row["assigned_value"]  # should be a dict

        if param_name == "hvac_params":
            # assigned_val is a dict with e.g.:
            # {
            #   "heating_day_setpoint": 20.2788,
            #   "heating_day_setpoint_range": (19.0,21.0),
            #   "cooling_day_setpoint": 24.55,
            #   "schedule_details": {...},
            #   etc.
            # }
            for k, v in assigned_val.items():
                building_rows.append({
                    "ogc_fid": bldg_id,
                    "param_name": k,
                    "param_value": v
                })

        elif param_name == "zones":
            # assigned_val is a dict with zone_name => { "hvac_object_name": "...", ... }
            for zone_name, zone_dict in assigned_val.items():
                # zone_dict might be something like:
                # {
                #   "hvac_object_name": "Zone1 Ideal Loads",
                #   "hvac_object_type": "ZONEHVAC:IDEALLOADSAIRSYSTEM",
                #   ...
                # }
                for zparam, zval in zone_dict.items():
                    zone_rows.append({
                        "ogc_fid": bldg_id,
                        "zone_name": zone_name,
                        "param_name": zparam,
                        "param_value": zval
                    })

        else:
            # If there's any other param_name, skip or handle differently
            pass

    # Convert to DataFrame
    df_build = pd.DataFrame(building_rows)
    df_zone = pd.DataFrame(zone_rows)

    # Optional: reorder columns
    df_build = df_build[["ogc_fid", "param_name", "param_value"]]
    df_zone = df_zone[["ogc_fid", "zone_name", "param_name", "param_value"]]

    # Write to CSV
    df_build.to_csv(out_build_csv, index=False)
    df_zone.to_csv(out_zone_csv, index=False)

    print(f"[INFO] Wrote building-level picks to {out_build_csv}")
    print(f"[INFO] Wrote zone-level picks to {out_zone_csv}")


def main():
    # 1) Path to your original HVAC CSV file
    csv_in = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_params.csv"

    # 2) Paths for output CSVs
    csv_build_out = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_building.csv"
    csv_zone_out = r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_zones.csv"

    # 3) Load the CSV
    df_assigned = pd.read_csv(csv_in)

    # 4) Convert assigned_value from string to dict
    df_assigned["assigned_value"] = df_assigned["assigned_value"].apply(parse_assigned_value)

    # 5) Flatten
    flatten_hvac_data(
        df_input=df_assigned,
        out_build_csv=csv_build_out,
        out_zone_csv=csv_zone_out
    )


if __name__ == "__main__":
    main()


[INFO] Wrote building-level picks to D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_building.csv
[INFO] Wrote zone-level picks to D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_zones.csv


[INFO] Wrote building-level picks to D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_building.csv
[INFO] Wrote zone-level picks to D:\Documents\E_Plus_2030_py\output\assigned\assigned_hvac_zones.csv


In [23]:
import pandas as pd
import ast  # for safely parsing a tuple string like "(4.0, 5.0)"


def transform_dhw_log_to_structured(
    csv_input="output/assigned/assigned_dhw_params.csv",
    csv_output="output/assigned/structured_dhw_params.csv",
):
    """
    Reads the 'flat' DHW CSV with columns (ogc_fid, param_name, assigned_value).
    Produces a 'structured' CSV with columns:
       ogc_fid, sub_key, eplus_object_type, eplus_object_name,
       param_name, param_value, param_min, param_max

    Key logic:
      - If param_name ends with "_range", parse min/max from assigned_value
        and store them in memory.
      - If param_name is the same as the range version minus "_range",
        unify them in one row => param_value + param_min + param_max.
      - For E+ objects (like "dhw_waterheater.obj_type"), store them
        so we can fill eplus_object_type/eplus_object_name in the final row.
    """

    df = pd.read_csv(csv_input)

    # We'll keep data in a nested dictionary:
    # final_dict[(ogc_fid, sub_key, base_param)] = {
    #    "value": <float or str>,
    #    "min": <float or None>,
    #    "max": <float or None>,
    #    "obj_type": <str or None>,
    #    "obj_name": <str or None>
    # }
    final_dict = {}

    def get_subdict(fid, s_key, base_param):
        if (fid, s_key, base_param) not in final_dict:
            final_dict[(fid, s_key, base_param)] = {
                "value": None,
                "min": None,
                "max": None,
                "obj_type": None,
                "obj_name": None
            }
        return final_dict[(fid, s_key, base_param)]

    for i, row in df.iterrows():
        # Must have these columns
        if "ogc_fid" not in row or "param_name" not in row or "assigned_value" not in row:
            continue

        ogc_fid = row["ogc_fid"]
        param_name = str(row["param_name"])
        assigned_value = row["assigned_value"]

        # We'll skip non-DHW if your CSV might have geometry/hvac. 
        # Or you can remove this if everything is DHW:
        if not param_name.startswith("dhw_"):
            # We'll keep it here in case you only want to transform "dhw_*"
            pass

        # 1) If we see something like "dhw_waterheater.obj_type" => E+ object type
        #    or "dhw_waterheater.Name" => E+ object name
        # We'll parse sub_key = "dhw_waterheater" and field = "obj_type" or "Name".
        # Then we store them so we can fill them in final output.

        # 2) If we see something like "occupant_density_m2_per_person_range",
        #    we parse the range => (minVal, maxVal).
        # 3) If we see "occupant_density_m2_per_person", that's the final pick.

        # Let's parse the sub_key from param_name, if there's a dot, e.g. 
        # "dhw_waterheater.obj_type" => sub_key="dhw_waterheater", field="obj_type"
        # If no dot => sub_key = "dhw", field = rest?

        sub_key = "dhw_top"  # default if we don't find anything
        field = param_name
        if '.' in param_name:
            # e.g. "dhw_waterheater.obj_type"
            sub_key, field = param_name.rsplit('.', 1)  # split from right

        if field.endswith("_range"):
            # e.g. "occupant_density_m2_per_person_range"
            base_param = field[:-6]  # remove "_range"
            subd = get_subdict(ogc_fid, sub_key, base_param)

            # parse assigned_value => (minVal, maxVal)
            # e.g. "(27.0, 33.0)"
            try:
                tval = ast.literal_eval(str(assigned_value))
                if isinstance(tval, (list, tuple)) and len(tval) == 2:
                    subd["min"], subd["max"] = tval
            except:
                pass

        elif field in ("obj_type", "Name"):
            # It's an E+ object pointer
            # store them in subdict => base_param = "EPOBJ" or something
            # Actually, let's define base_param = "" so we handle it as an object-level thing
            base_param = "EPOBJ"
            subd = get_subdict(ogc_fid, sub_key, base_param)

            if field == "obj_type":
                subd["obj_type"] = assigned_value
            else:  # field=="Name"
                subd["obj_name"] = assigned_value

        else:
            # normal param => occupant_density_m2_per_person
            # or "dhw_daily_liters"
            base_param = field
            subd = get_subdict(ogc_fid, sub_key, base_param)

            # try to interpret as float
            subd["value"] = assigned_value

    # Now we have a dict that merges range with final value, object type, etc.
    # We'll produce final rows with columns we want:
    structured_rows = []
    for (fid, s_key, base_param), valdict in final_dict.items():
        # If base_param == "EPOBJ", that means it's the object reference (obj_type, obj_name)
        # Usually, we want to store them in param_name => "obj_type" or "object_name".
        # But let's unify everything in "param_name, param_value".
        # We handle that scenario if the user wants an actual param row or skip it.
        # We'll separate them out below.

        # If the user wants a single param row for occupant_density, occupant_density_range, etc.:
        if base_param == "EPOBJ":
            # This indicates the sub_key is an E+ object prefix like "dhw_waterheater"
            # valdict might have obj_type => "WATERHEATER:MIXED" and obj_name => "MyDHW_0_WaterHeater"
            # We'll store them as "eplus_object_type" and "eplus_object_name" so that
            # *other* param rows can refer to them.

            # we won't produce a row for EPOBJ itself, because it doesn't correspond
            # to a normal param. We'll store it so that we can fill in the final eplus_object_type
            # eplus_object_name for all rows with sub_key = s_key, if we want.
            pass
        else:
            # It's a normal param => occupant_density_m2_per_person, setpoint_c, etc.
            param_name = base_param
            param_val = valdict["value"]
            param_min = valdict["min"]
            param_max = valdict["max"]

            # We'll parse them as floats if possible:
            def try_float(x):
                try:
                    return float(x)
                except:
                    return x

            param_val = try_float(param_val)
            param_min = try_float(param_min)
            param_max = try_float(param_max)

            # eplus object type & name if we find them from "EPOBJ"
            # let's see if there's a subdict with base_param="EPOBJ" for this sub_key
            ep_obj = final_dict.get((fid, s_key, "EPOBJ"), {})
            eplus_type = ep_obj.get("obj_type", None)
            eplus_name = ep_obj.get("obj_name", None)

            structured_rows.append({
                "ogc_fid": fid,
                "sub_key": s_key,
                "eplus_object_type": eplus_type,
                "eplus_object_name": eplus_name,
                "param_name": param_name,
                "param_value": param_val,
                "param_min": param_min,
                "param_max": param_max
            })

    # Additionally, we might want a row for the object-level fields (obj_type, obj_name).
    # e.g., param_name="obj_type" => param_value="WATERHEATER:MIXED"
    #        param_name="Name" => param_value="MyDHW_0_WaterHeater"
    # We'll do a second pass for those "EPOBJ" entries if you want them each as a row.

    object_rows = []
    for (fid, s_key, base_param), valdict in final_dict.items():
        if base_param == "EPOBJ":
            # It's an object definition => let's produce 2 param rows:
            eplus_type = valdict["obj_type"]
            eplus_name = valdict["obj_name"]

            # param_name="obj_type"
            object_rows.append({
                "ogc_fid": fid,
                "sub_key": s_key,
                "eplus_object_type": eplus_type,   # we can store it again
                "eplus_object_name": eplus_name,   # or keep these blank
                "param_name": "obj_type",
                "param_value": eplus_type,
                "param_min": None,
                "param_max": None
            })
            # param_name="Name"
            object_rows.append({
                "ogc_fid": fid,
                "sub_key": s_key,
                "eplus_object_type": eplus_type,
                "eplus_object_name": eplus_name,
                "param_name": "Name",
                "param_value": eplus_name,
                "param_min": None,
                "param_max": None
            })

    # Combine them
    all_rows = structured_rows + object_rows
    df_struct = pd.DataFrame(all_rows)

    # Sort rows so the object-level lines appear first or last. Not strictly needed:
    df_struct.sort_values(by=["ogc_fid", "sub_key", "param_name"], inplace=True)

    df_struct.to_csv(csv_output, index=False)
    print(f"[transform_dhw_log_to_structured] => Wrote structured CSV to {csv_output}")


if __name__ == "__main__":
    transform_dhw_log_to_structured(
        csv_input=r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_dhw_params.csv",
        csv_output=r"D:\Documents\E_Plus_2030_py\output\assigned\assigned_dhw_params.csv"
    )


[transform_dhw_log_to_structured] => Wrote structured CSV to D:\Documents\E_Plus_2030_py\output\assigned\assigned_dhw_params.csv
